<a href="https://colab.research.google.com/github/ssvadla/Research_01/blob/main/EM_GMM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn import datasets
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.datasets import make_classification
from sklearn.calibration import CalibratedClassifierCV
import os
import pandas as pd 
import numpy as np
from nltk.stem import WordNetLemmatizer
import nltk
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from google.colab import drive


drive.mount('/content/drive')


nltk.download('stopwords')
nltk.download('wordnet')
stopword=nltk.corpus.stopwords.words('english')
wl= WordNetLemmatizer()

def clean_text(text):
  text="".join([word.lower() for word in text if word not in string.punctuation])
  tokens = re.split('\W+',text)
  text = [wl.lemmatize(word) for word in tokens if word not in stopword]
  return text

def load_data():

    train1 = pd.read_csv('/content/drive/My Drive/Research/train_data1.csv')
    train2 = pd.read_csv('/content/drive/My Drive/Research/train_data2.csv')
    train3 = pd.read_csv('/content/drive/My Drive/Research/train_data3.csv')
    train4 = pd.read_csv('/content/drive/My Drive/Research/train_data4.csv')
    train5 = pd.read_csv('/content/drive/My Drive/Research/train_data5.csv')
    train6 = pd.read_csv('/content/drive/My Drive/Research/train_data6.csv')
    train7 = pd.read_csv('/content/drive/My Drive/Research/train_data7.csv')
    train8 = pd.read_csv('/content/drive/My Drive/Research/train_data8.csv')
    train9 = pd.read_csv('/content/drive/My Drive/Research/train_data9.csv')
    train10 = pd.read_csv('/content/drive/My Drive/Research/train_data10.csv')
    train_highKappa = pd.read_csv('/content/drive/My Drive/Research/train_data_highkappa.csv')

    train = train1
    train_list = [train2,train3,train4,train5,train6,train7,train8,train9,train10,train_highKappa]
    for i in train_list:
        print(i)
        train = train.append(i)

    train.sort_values("Sentence", inplace = True)
    new_train = train.drop_duplicates(subset ="Sentence")
    train = new_train
    train['Target']=train['Target'].replace(['Others'],'Invalid')
    print("all data shape @@@@@@@@@@",train.shape)
    print(type(train))
    print(train['Target'].unique())
    tfidf_vect = TfidfVectorizer(analyzer = clean_text)
    X_tfidf = tfidf_vect.fit_transform(train['Sentence'])
    print("Shape of the vectyor",X_tfidf.shape)
    test = pd.read_csv(r'/content/drive/My Drive/Research/test_data.csv')
    test['Target']=test['Target'].replace(['Others'],'Invalid')
    test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
    test['Sentence'] = test['Sentence'].str.replace('[^\w\s]','') 
    words = stopwords.words('english')
    test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x for x in x.split() if x not in words))
    t_p = tfidf_vect.transform(test['Sentence'])
	#X_train, x_val, Y_train, y_val = train_test_split(X_tfidf,train['Target'],test_size=0.26,random_state=42)
    # print("Done")
    return X_tfidf, train['Target'],train
  
X, y , train = load_data()
y = LabelEncoder().fit_transform(y)

print(type(X))
print(type(y))
#print(X.columns,y.columns)

# X = train['Sentence']
# y = train['Target']



# iris = datasets.load_iris()

# X = pd.DataFrame(iris.data)
# X.columns = ['Sepal_Length', 'Sepal_Width', 'Petal_Length', 'Petal_Width']

y = pd.DataFrame(y)
y.columns = ['target']

print("###########",y.columns)

# Build the KMeans Model
kmeans = KMeans(n_clusters=6)
clusters = kmeans.fit_predict(X)

from scipy.stats import mode
labels = np.zeros_like(clusters)

for i in range(6):
    cat = (clusters == i)
    labels[cat] = mode(y.target[cat])[0]
    
acc = accuracy_score(y.target, labels)
print('Accuracy = ', acc)

# plt.figure(figsize=(10, 10))
# colormap = np.array(['red', 'lime', 'blue','yellow', 'green', 'black'])

# # Plot the Original Classifications using Petal features
# plt.subplot(2, 2, 1)
# plt.scatter(X.text , c = colormap[y.Targets], s=40)
# plt.title('Real Clusters')
# plt.xlabel('Petal Length')
# plt.ylabel('Petal Width')

# # Plot KMeans Model Classifications
# plt.subplot(2, 2, 2)
# plt.scatter(X.Petal_Length, X.Petal_Width, c = colormap[labels], s = 40)
# plt.title('KMeans Clusters')
# plt.xlabel('Petal Length')
# plt.ylabel('Petal Width')

# General EM for GMM
from sklearn import preprocessing

X,y , train = load_data()
y = LabelEncoder().fit_transform(y)

y = pd.DataFrame(y)
y.columns = ['target']

train['Target'] = LabelEncoder().fit_transform(train['Target'] )
print("tye ----------",type(X))
hello = pd.DataFrame(X.toarray())
print(type(y))

# transform data such that the distribution mean = 0 and std = 1
scaler = preprocessing.StandardScaler(with_mean=False)
scaler.fit(hello)
scaled_X = scaler.transform(hello)

xs = pd.DataFrame(scaled_X) #, columns = X.columns)

from sklearn.mixture import GaussianMixture

gmm = GaussianMixture(n_components=6)
gmm_y = gmm.fit_predict(xs)

labels = np.zeros_like(clusters)

for i in range(6):
    cat = (gmm_y == i)
    labels[cat] = mode(y.target[cat])[0]
    
acc = accuracy_score(y.target, labels)
print("Accuracy using GMM = ", acc)





# plt.subplot(2, 2, 3)
# plt.scatter(X.text ,c = colormap[gmm_y], s = 40)
# plt.subplots_adjust(hspace=0.4, wspace=0.4)
# plt.title('GMM Clusters')
# plt.xlabel('Petal Length')
# plt.ylabel('Petal Width')

In [11]:
def load_data():

    train1 = pd.read_csv('/content/drive/My Drive/Research/train_data1.csv')
    train2 = pd.read_csv('/content/drive/My Drive/Research/train_data2.csv')
    train3 = pd.read_csv('/content/drive/My Drive/Research/train_data3.csv')
    train4 = pd.read_csv('/content/drive/My Drive/Research/train_data4.csv')
    train5 = pd.read_csv('/content/drive/My Drive/Research/train_data5.csv')
    train6 = pd.read_csv('/content/drive/My Drive/Research/train_data6.csv')
    train7 = pd.read_csv('/content/drive/My Drive/Research/train_data7.csv')
    train8 = pd.read_csv('/content/drive/My Drive/Research/train_data8.csv')
    train9 = pd.read_csv('/content/drive/My Drive/Research/train_data9.csv')
    train10 = pd.read_csv('/content/drive/My Drive/Research/train_data10.csv')
    train_highKappa = pd.read_csv('/content/drive/My Drive/Research/train_data_highkappa.csv')

    train = train1
    train_list = [train2,train3,train4,train5,train6,train7,train8,train9,train10,train_highKappa]
    for i in train_list:
        print(i)
        train = train.append(i)

    train.sort_values("Sentence", inplace = True)
    new_train = train.drop_duplicates(subset ="Sentence")
    train = new_train
    train['Target']=train['Target'].replace(['Others'],'Invalid')
    print("all data shape @@@@@@@@@@",train.shape)
    print(type(train))
    print(train['Target'].unique())
    tfidf_vect = TfidfVectorizer(analyzer = clean_text)
    X_tfidf = tfidf_vect.fit_transform(train['Sentence'])
    print("Shape of the vectyor",X_tfidf.shape)
    test = pd.read_csv(r'/content/drive/My Drive/Research/test_data.csv')
    test['Target']=test['Target'].replace(['Others'],'Invalid')
    test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
    test['Sentence'] = test['Sentence'].str.replace('[^\w\s]','') 
    words = stopwords.words('english')
    test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x for x in x.split() if x not in words))
    t_p = tfidf_vect.transform(test['Sentence'])
	#X_train, x_val, Y_train, y_val = train_test_split(X_tfidf,train['Target'],test_size=0.26,random_state=42)
    # print("Done")
    return X_tfidf, train['Target'],train,t_p,test

In [ ]:
X_tfidf, train['Target'],train,t_p,test = load_data()

In [15]:
test_target = test['Target']

Index(['Unnamed: 0', 'Sentence', 'Target'], dtype='object')

In [7]:
test_df = pd.DataFrame(t_p.toarray())

In [8]:
gmm_test_pred = gmm.predict(test_df)

In [ ]:
test = pd.DataFrame(test)
test.columns = ['target']

In [18]:
len(gmm_test_pred)

521

In [19]:
len(cat)

521

In [22]:
len(test_labels)

521

In [21]:
test_labels = np.zeros_like(gmm_test_pred)

In [24]:
test_target = LabelEncoder().fit_transform(test_target)

In [26]:

#test_labels = np.zeros_like(clusters)

for i in range(6):
    cat = (gmm_test_pred == i)
    test_labels[cat] = mode(test_target[cat])[0]
    
acc = accuracy_score(test_target, test_labels)
print("Accuracy using GMM = ", acc)

Accuracy using GMM =  0.5124760076775432


In [27]:
f1_Score_metric = f1_score(test_target,test_labels,average='weighted')
print("F1 score ",f1_Score_metric)

F1 score  0.347287040735797


In [ ]:
f1_Score_metric = f1_score(test['Target'],test_pred,average='weighted')
print("F1 score ",f1_Score_metric)